# How the backpropagation algorithm works

In Chapter 1 we saw how neural networks can learn their weights and biases using the gradient descent algorithm, but we didn't discuss how to compute the gradient of the cost function!

The backpropagation algorithm was originally introduced in the 70s, but not until a [famous paper](https://www.academia.edu/2520405/Learning_representations_by_back-propagating_errors?auto=download) by Rumelhart, Hinton and Williams described several neural networks where backpropagation works way faster than earlier approaches to learning, making it possible to use neural nets to solve previously insoluble problems.

This chapter is more mathematically involved than the rest of the book.

At the heart of backpropagation is an expression for the partial derivative of the cost function C with respect to any weight w or bias b in the network. And while the expression is complex, it also has a beauty to it, with each element having a natural, intuitive interpretation.

Backpropagation isn't just a fast algorithm for learning. It actually gives us detailed insights into how changing the weights and biases changes the overall behaviour of the network.

## Warm up

Let's begin with a notation which lets us refer to weights in the network in an unambiguous way. We'll use $w^{l}_{jk}$ to denote the weight for the connection from the $k^{th}$ neuron in the $(l-1)^{th}$ layer to the $j^{th}$ neuron in the $l^{th}$ layer.

We use a similar notation for the network's biases and activations. Let's use $b^{l}_{j}$ for the bias in the $j^{th}$ neuron in the $l^{th}$ layer. 

And we use $a^{l}_{j}$ for the activation in the $j^{th}$ neuron in the $l^{th}$ layer. Remember that the activation is the result of the neuron.

With these notations, the activation $a^{l}_{j}$ of the $j^{th}$ neuron in the $l^{th}$ layer is related to the activations in the $(l-1)^{th}$ layer by:

$$a^{l}_{j}=\sigma(\sum_{k}w^{l}_{jk}a^{l-1}_{j}+b^{l}_{j})$$

where the sum is over all neurons `k` in the $(l-1)^{th}$ layer.

To rewrite this expression in a matrix form we define a *weight matrix* $w^{l}$ for each layer `l`, where the entries are just the weights connecting the $l^{th}$ layer of neurons: the entry in the $j^{th}$ row and the $k^{th}$ column is $w^{l}_{jk}$.

Similarly, for each layer `l` we define a *bias vector* $b^{l}$, where the components are just the values $b^{l}_{j}$, one component for each neuron in `l`.

We also define an *activation vector* $a^{l}$, where the components are just the values $a^{l}_{j}$, one component for each neuron in `l`.

One last thing is to treat $\sigma$ as a vector in the sense of applying the function to each component of the vector we are applying the function to.

With these notations in mind, the previous equation can be rewritten as

$$a^{l} = \sigma(w^{l}a^{l-1} + b^{l}) $$

This expression gives us a much more global way of thinking about how the activations in one layer relate to activations in the previous layer: we just apply the weight matrix to the activations, then add the bias vector, and finally apply the $\sigma$ function.

The global view is often easier and more succint than the neuron-by-neuron view we've taken to now. Think of it as a way of escaping index hell, while remaining precise about what is going on. The expression is also useful, because most matrix libraries provide fast ways of implementing matrix multiplication, vector addition, and vectorization.

When using this equation to compute $a^{l}$, we compute the intermediate quantity $z^{l}\equiv w^{l}a^{l-1} + b^{l}$ along the way, what we call the *weighted input* to the neurons in layer `l`, with components $z^{l}_{j}=\sum_{k}w^{l}_{jk}a^{l-1}_{j}+b^{l}_{j}$.

## Two assumptions

The goal of backpropagation is to compute the partial derivatives $\frac{\partial C}{\partial w}$ and $\frac{\partial C}{\partial b}$ of the cost function C with respect to any weight `w` or bias `b` in the network.

For backpropagation to work we need to make two main assumptions about the form of the cost function. Let's have the quadratic cost function in mind as an example:

$$ C = \frac{1}{2n}\sum_{x} || y(x) - a^{L}(x) ||^{2} $$

where `n` is the total number of training examples, the sum is over each training example `x`, `y(x)` is the corresponding desired output, `L` the number of layers in the network, and $a^{L}(x)$ is the vector of activations output from the network when x is input.

### C can be written as an average over cost functions for individual training examples

$$ C = \frac{1}{n}\sum_{x} C_{x} $$

The reason we need this assumption is because what backpropagation actually let us do is compute the partial derivatives $\frac{\partial C_{x}}{\partial w}$ and $\frac{\partial C_{x}}{\partial b}$ for a single training example, and then we recover $\frac{\partial C}{\partial w}$ and $\frac{\partial C}{\partial b}$ by averaging over all training examples.

### C can be written as a function of the outputs from the neural network

$$ C = C(a^{L}) $$

## The Hadamard product

The backpropagation algorithm is based on common linear algebraic operations. But one is a little less common used, the __Hadamard Product__. We use  $s \circ t$ to denote the element wise product of two vectors, as opposed to the dot product.

Thus, the components of $s \circ t$ are just $(s \circ t)_{j} = s_{j}t_{j} $

## Four fundamental equations behind backpropagation

Backpropagation is about understanding how changing the weights and biases in a network changes the cost function. Ultimately, this means computing the partial derivatives $\frac{\partial C}{\partial w^{l}_{jk}}$ and $\frac{\partial C}{\partial b^{l}_{j}}$. But to compute those, we first introduce an intermediate quantity $\delta^{l}_{j}$, the *error* in the $j^{th}$ neuron in the $l^{th}$ layer.

Backpropagation will give us a procedure to compute the error and then will relate to the partial derivatives we are looking for.

We define the error $\delta^{l}_{j}$ as 

$$ \delta^{l}_{j} \equiv \frac{\partial C}{\partial z^{l}_{j}} $$

Backpropagation is based around 4 fundamental equations. Together, those equations give us a way of computing both the error $\delta^{l}$ and the gradient of the cost function.

### BP1 - An equation for the error in the output layer

$$\delta^{l}_{j} = \frac{\partial C}{\partial a^{l}_{j}}\sigma'(z^{L}_{j}) \tag{BP1}$$

This is a very natural expression: the derivative measures how fast the cost is changing as a functon of the j-th output activation. If C doesn't depend much on a particular output neuron, this derivative will be small and thus $\delta^{l}_{j}$ will be small. The second term, $ \sigma'(z^{L}_{j}) $, measures how fast the activation function is changing at $z^{L}_{j}$.

Notice that everything is easily computed; in particular, $z^{L}_{j}$ is already been computed, so it's only a small additional overhead to compute $\sigma'(z^{L}_{j})$. The first term will of course depend on the form of the cost function, but assuming that this function C is known, there should be little trouble computing it.

Equation BP1 is a componentwise expression for $\delta^{l}_{j}$. It's easy to rewrite the equation in a matrix-based form as

$$ \delta^{l} = \nabla_{a} C \circ \sigma'(z^{L}) \tag{BP1a}$$

### BP2 - An equation for the error in terms of the error in the next layer

$$ \delta^{l} = ((w^{l + 1})^{T} \delta^{l+1}) \circ\sigma'(z^{l}) \tag{BP2}$$

where $(w^{l + 1})^{T}$ is the transpose of the weight matrix $w^{l + 1}$ for the $(l + 1)^{th}$ layer.

This may appear complicated, but it has a nice interpretation. Suppose we know the error $\delta^{l+1}$ at the $(l + 1)^{th}$ layer. When we apply the transpose weight matrix, we can think intuitively of this as moving the error *backward* through the network, giving us some sort of measure of the error at the output of the $l^{th}$ layer. We then take the Hadamard product, whic moves the error backward through the activation function in layer `l`, giving us the error in the weighted input to layer `l`.

By combining BP1 and BP2, we can compute the error $\delta^{l}$ for any layer in the network. We start by using BP1 to compute $\delta^{l}$, then apply BP2 again and again all the way back through the network.


### BP3 - An equation for the rate of change of the cost with respect to any bias in the network

$$\frac{\partial C}{\partial b^{l}_{j}} =  \delta^{l}_{j} \tag{BP3}$$

That is, the error is *exactly equal* to the rate of change. This is good news since BP1 and BP2 have already told us how to compute $\delta^{l}_{j}$. We can rewrite this as:

$$\frac{\partial C}{\partial b} =  \delta \tag{BP3a}$$

where it is understood that $\delta$ is being evaluated at the same neuron as the bias `b`.

### BP4 - An equation for the rate of change of the cost with respect to any weight in the network

$$\frac{\partial C}{\partial w^{l}_{jk}} = a^{l-1}_{k}\delta^{l}_{j} \tag{BP4}$$